In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

In [6]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    print(model_output)
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3471,  0.3033,  0.5519,  ...,  1.3244,  0.5176, -0.2550],
         [ 1.2137,  0.4327,  1.7353,  ...,  1.2354,  1.3629,  1.1924],
         [ 0.7118,  0.4431,  0.8161,  ...,  1.0828,  2.6126,  0.9954],
         [ 0.4893,  0.8175,  0.3924,  ..., -0.1170,  0.5501, -0.0983],
         [ 0.0468,  0.3585,  0.4143,  ...,  0.9164,  0.5802,  0.0154]]]), pooler_output=tensor([[ 0.3941, -0.0054, -0.1369,  0.0412,  0.1984, -0.4276,  0.1191,  0.1195,
         -0.4111, -0.3154,  0.3484, -0.2489, -0.1621, -0.3853, -0.3756,  0.0895,
         -0.1327, -0.1038,  0.2586,  0.2757,  0.0181,  0.0439, -0.5461, -0.1934,
         -0.2076, -0.2244,  0.3835,  0.0842,  0.5532, -0.2212,  0.0552,  0.4834,
          0.3691,  0.1255,  0.3814,  0.5966, -0.1828,  0.0155, -0.2545,  0.7083,
         -0.4018,  0.1376, -0.1788, -0.3726, -0.3718, -0.6064, -0.4973, -0.0948,
          0.0827,  0.3623,  0.0893,  0.3826,  0.3928,  0.2825, -0.2595,  0.2158